In [ ]:
TAIL = 0  # решка
HEAD = 1  # орел

n = int(input().strip())
if n == 1:
    print("0.000000 1.000000 0.000000")
    exit(0)

sequence = [
    [ [0] * 1 for _ in range(2) ],  
    [ [0] * 1 for _ in range(2) ]   
]
sequence[HEAD][1][0] = 1  
sequence[TAIL][0][0] = 1  

for length in range(1, n):
    new_sequence = [
        [ [0] * (length + 1) for _ in range(length + 2) ],
        [ [0] * (length + 1) for _ in range(length + 2) ]
    ]

    
    for last in (TAIL, HEAD):
        for k_val in range(length + 1):    
            for y_val in range(length):    
                cnt = sequence[last][k_val][y_val]
                if cnt == 0:
                    continue

                for new_char in (TAIL, HEAD):
                    nlast = new_char
                    nk = k_val + (1 if new_char == HEAD else 0)
                    ny = y_val

                    if last == HEAD and new_char == HEAD:
                        ny += 1

                    new_sequence[nlast][nk][ny] += cnt

    sequence = new_sequence

total_seqs = 2 ** n  
win_A = 0  
win_B = 0 
draw = 0 

for last in (TAIL, HEAD):
    for k_val in range(n + 1):     
        for y_val in range(n):     
            cnt = sequence[last][k_val][y_val]
            if cnt == 0:
                continue
            L = 1 if last == HEAD else 0
            diff = k_val - L - 2 * y_val

            if diff > 0:
                win_A += cnt
            elif diff < 0:
                win_B += cnt
            else:
                draw += cnt

pA = win_A / total_seqs
pDraw = draw / total_seqs
pB = win_B / total_seqs


print(f"{pA:.6f} {pDraw:.6f} {pB:.6f}")

2
[[[1], [0]], [[0], [1]]]
0.250 0.500 0.250


In [ ]:
import json

json_data = """{
	"serpset": [
		{
			"query": "okna",
		 	"results": [
				{"position": 0, "url": "okna-msk.ru", "relevance": 0.95, "cost": 77},
				{"position": 1, "url": "okna-pvh.ru", "relevance": 0.95, "cost": 70},
				{"position": 2, "url": "ne-okna.ru", "relevance": 0.3, "cost": 100},
				{"position": 3, "url": "best-okna.ru", "relevance": 0.1, "cost": 0}
		]},
		{
			"query": "lego",
			"results": [
				{"position": 0, "url": "lego.ru", "relevance": 0.95, "cost": 15},
				{"position": 1, "url": "lego-mir.ru", "relevance": 0.7, "cost": 30},
				{"position": 2, "url": "disney.ru", "relevance": 0.3, "cost": 100}]
		}],
	"new_documents": [
		{"query": "weather", "url": "yandex.ru/pogoda", "relevance": 1, "cost": 0},
		{"query": "okna", "url": "yandex.ru/okna", "relevance": 1, "cost": 100},
		{"query": "lego", "url": "yandex.ru/lego", "relevance": 0.4, "cost": 10}
	]}
"""
data_dict = json.loads(json_data)
print(data_dict)
# serpset - список размеченных поисковых выдач
# new_documents - размеченные документы

# Поисковая выдача — это упорядоченный список документов, показанных поисковой системой по текстовому запросу. Документы размечаются по релевантности и по максимальному количеству денег, которое документ может принести.
# Вставьте минимальное количество документов на выдачу, чтобы максимизировать деньги и не ухудшить качество. Размер выдачи по запросу тоже не должен измениться: новые документы вклиниваются в выдачу, а низ удаляется.
# Порядок изначальных документов нельзя менять. Выведите максимальную суммарную выручку по всем запросам.

{'serpset': [{'query': 'okna', 'results': [{'position': 0, 'url': 'okna-msk.ru', 'relevance': 0.95, 'cost': 77}, {'position': 1, 'url': 'okna-pvh.ru', 'relevance': 0.95, 'cost': 70}, {'position': 2, 'url': 'ne-okna.ru', 'relevance': 0.3, 'cost': 100}, {'position': 3, 'url': 'best-okna.ru', 'relevance': 0.1, 'cost': 0}]}, {'query': 'lego', 'results': [{'position': 0, 'url': 'lego.ru', 'relevance': 0.95, 'cost': 15}, {'position': 1, 'url': 'lego-mir.ru', 'relevance': 0.7, 'cost': 30}, {'position': 2, 'url': 'disney.ru', 'relevance': 0.3, 'cost': 100}]}], 'new_documents': [{'query': 'weather', 'url': 'yandex.ru/pogoda', 'relevance': 1, 'cost': 0}, {'query': 'okna', 'url': 'yandex.ru/okna', 'relevance': 1, 'cost': 100}, {'query': 'lego', 'url': 'yandex.ru/lego', 'relevance': 0.4, 'cost': 10}]}


In [ ]:
import math
import json
import sys

def count_bits(mask):
    return bin(mask).count('1')

def add_to_pareto(lst, new_pair):
    new_q, new_r = new_pair
    to_remove = []
    dominated = False
    for idx, (q, r) in enumerate(lst):
        if q >= new_q and r >= new_r:
            dominated = True
            break
        if new_q >= q and new_r >= r:
            to_remove.append(idx)
    if dominated:
        return lst
    new_list = [pair for idx, pair in enumerate(lst) if idx not in to_remove]
    new_list.append((new_q, new_r))
    return new_list


data = json.loads(json_data)
total_revenue = 0.0

for serp in data['serpset']:
    query = serp['query']
    D = sorted(serp['results'], key=lambda x: x['position'])
    M = [doc for doc in data['new_documents'] if doc['query'] == query]
    L = len(D)
    n_new = len(M)

    orig_quality = 0.0
    orig_revenue = 0.0
    for idx, doc in enumerate(D, start=1):
        orig_quality += doc['relevance'] / idx
        orig_revenue += doc['cost'] / math.sqrt(idx)

    best_rev = orig_revenue

    # sequence: [i][mask] = [(quality, revenue), ...]
    # i - num of placed original docs
    # mask - chosen new docs
    if n_new > 0:
        sequence = [[[] for _ in range(1 << n_new)] for _ in range(L + 1)]
        sequence[0][0] = [(0, 0)]

        for i in range(L + 1):
            for mask in range(1 << n_new):
                p_len = i + count_bits(mask)
                if p_len >= L:
                    continue

                for (q, r) in sequence[i][mask]:
                    if i < L:
                        pos = p_len + 1
                        doc = D[i]
                        new_q = q + doc['relevance'] / pos
                        new_r = r + doc['cost'] / math.sqrt(pos)
                        sequence[i + 1][mask] = add_to_pareto(sequence[i + 1][mask], (new_q, new_r))

                    for j in range(n_new):
                        if mask & (1 << j):
                            continue
                        pos = p_len + 1
                        doc = M[j]
                        new_q = q + doc['relevance'] / pos
                        new_r = r + doc['cost'] / math.sqrt(pos)
                        new_mask = mask | (1 << j)
                        sequence[i][new_mask] = add_to_pareto(sequence[i][new_mask], (new_q, new_r))


        for i in range(L + 1):
            for mask in range(1 << n_new):
                if i + count_bits(mask) != L:
                    continue
                for (q, r) in sequence[i][mask]:
                    if q >= orig_quality and r > best_rev:
                        best_rev = r

    total_revenue += best_rev

total_revenue = round(total_revenue, 2)
print(f"{total_revenue:.2f}")

338.81
